In [ ]:
from abc import ABC, abstractmethod

# Visitor Interface
class Visitor(ABC):
    @abstractmethod
    def visit_file(self, file):
        pass

    @abstractmethod
    def visit_directory(self, directory):
        pass

# Element Interface
class Element(ABC):
    @abstractmethod
    def accept(self, visitor):
        pass

# Concrete Element: File
class File(Element):
    def __init__(self, name, size):
        self.name = name
        self.size = size  # Size in bytes

    def accept(self, visitor):
        visitor.visit_file(self)

# Concrete Element: Directory
class Directory(Element):
    def __init__(self, name):
        self.name = name
        self.children = []  # Can contain Files or Directories

    def add(self, element):
        self.children.append(element)

    def accept(self, visitor):
        visitor.visit_directory(self)

# Concrete Visitor: SizeCalculator
class SizeCalculator(Visitor):
    def __init__(self):
        self.total_size = 0

    def visit_file(self, file):
        print(f"Calculating size of file: {file.name} ({file.size} bytes)")
        self.total_size += file.size

    def visit_directory(self, directory):
        print(f"Entering directory: {directory.name}")
        for child in directory.children:
            child.accept(self)
        print(f"Exiting directory: {directory.name}")

# Concrete Visitor: DisplayVisitor
class DisplayVisitor(Visitor):
    def __init__(self):
        self.indent_level = 0

    def visit_file(self, file):
        print('    ' * self.indent_level + f"- File: {file.name} ({file.size} bytes)")

    def visit_directory(self, directory):
        print('    ' * self.indent_level + f"+ Directory: {directory.name}")
        self.indent_level += 1
        for child in directory.children:
            child.accept(self)
        self.indent_level -= 1


In [4]:
# Client Code
def main():
    # Create files
    file1 = File("file1.txt", 1000)
    file2 = File("file2.txt", 2000)
    file3 = File("file3.jpg", 5000)
    file4 = File("file4.mp3", 3000)

    # Create directories and build the structure
    root = Directory("root")
    root.add(file1)
    root.add(file2)

    images = Directory("images")
    images.add(file3)
    root.add(images)

    music = Directory("music")
    music.add(file4)
    root.add(music)

    # Create visitors
    size_calculator = SizeCalculator()
    display_visitor = DisplayVisitor()

    print("\n--- Displaying File Structure ---")
    root.accept(display_visitor)

    print("\n--- Calculating Total Size ---")
    root.accept(size_calculator)
    print(f"\nTotal size: {size_calculator.total_size} bytes")

if __name__ == "__main__":
    main()



--- Displaying File Structure ---
+ Directory: root
    - File: file1.txt (1000 bytes)
    - File: file2.txt (2000 bytes)
    + Directory: images
        - File: file3.jpg (5000 bytes)
    + Directory: music
        - File: file4.mp3 (3000 bytes)

--- Calculating Total Size ---
Entering directory: root
Calculating size of file: file1.txt (1000 bytes)
Calculating size of file: file2.txt (2000 bytes)
Entering directory: images
Calculating size of file: file3.jpg (5000 bytes)
Exiting directory: images
Entering directory: music
Calculating size of file: file4.mp3 (3000 bytes)
Exiting directory: music
Exiting directory: root

Total size: 11000 bytes


In [5]:
from abc import ABC, abstractmethod

# Visitor Interface
class ExpressionVisitor(ABC):
    @abstractmethod
    def visit_number(self, number):
        pass

    @abstractmethod
    def visit_addition(self, addition):
        pass

    @abstractmethod
    def visit_multiplication(self, multiplication):
        pass

# Element Interface
class Expression(ABC):
    @abstractmethod
    def accept(self, visitor):
        pass

# Concrete Element: Number
class Number(Expression):
    def __init__(self, value):
        self.value = value

    def accept(self, visitor):
        visitor.visit_number(self)

# Concrete Element: Addition
class Addition(Expression):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def accept(self, visitor):
        visitor.visit_addition(self)

# Concrete Element: Multiplication
class Multiplication(Expression):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def accept(self, visitor):
        visitor.visit_multiplication(self)

# Concrete Visitor: EvaluationVisitor
class EvaluationVisitor(ExpressionVisitor):
    def __init__(self):
        self.stack = []

    def visit_number(self, number):
        print(f"Visiting Number: {number.value}")
        self.stack.append(number.value)

    def visit_addition(self, addition):
        print("Visiting Addition")
        addition.left.accept(self)
        addition.right.accept(self)
        right = self.stack.pop()
        left = self.stack.pop()
        result = left + right
        print(f"Adding {left} + {right} = {result}")
        self.stack.append(result)

    def visit_multiplication(self, multiplication):
        print("Visiting Multiplication")
        multiplication.left.accept(self)
        multiplication.right.accept(self)
        right = self.stack.pop()
        left = self.stack.pop()
        result = left * right
        print(f"Multiplying {left} * {right} = {result}")
        self.stack.append(result)

In [6]:
# Client Code
def main():
    # Represent the expression: (5 + 3) * 2
    expr = Multiplication(
        Addition(Number(5), Number(3)),
        Number(2)
    )

    evaluator = EvaluationVisitor()
    expr.accept(evaluator)
    result = evaluator.stack.pop()
    print(f"\nResult of the expression: {result}")

if __name__ == "__main__":
    main()


Visiting Multiplication
Visiting Addition
Visiting Number: 5
Visiting Number: 3
Adding 5 + 3 = 8
Visiting Number: 2
Multiplying 8 * 2 = 16

Result of the expression: 16
